# Music Recommendation System

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('dataset.csv')
data.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [3]:
data.info()
data['track_genre'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
       'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk',
       'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie-pop', 'indie', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop-film', 'pop',
       'pow

In [4]:
df = data[['artists','album_name','track_name','track_genre',]]
df.head()

,artists,album_name,track_name,track_genre
0,Gen Hoshino,Comedy,Comedy,acoustic
1,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,acoustic
2,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,acoustic
3,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,acoustic
4,Chord Overstreet,Hold On,Hold On,acoustic


In [5]:
df.isnull().sum()
df.dropna(inplace=True)

/var/folders/b4/d7d_4c_13mb24yqk0885smv80000gn/T/ipykernel_7173/4081141694.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [6]:
df.isnull().sum()

artists        0
album_name     0
track_name     0
track_genre    0
dtype: int64

In [7]:
df.duplicated().sum()
df = df.drop_duplicates()

In [8]:

df['artists'] = df['artists'].apply(lambda x: "".join(x.split()).lower())
df['track_genre'] = df['track_genre'].apply(lambda x: x.lower())
df['track_name'] = df['track_name'].apply(lambda x: x.lower())
df['album_name'] = df['album_name'].apply(lambda x: x.lower())

In [9]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [10]:
def removepot(col):
    new_tag = []
    for i in col.split():
        new_tag.append(ps.stem(i))
    return " ".join(new_tag)

In [11]:
df.head()

,artists,album_name,track_name,track_genre
0,genhoshino,comedy,comedy,acoustic
1,benwoodward,ghost (acoustic),ghost - acoustic,acoustic
2,ingridmichaelson;zayn,to begin again,to begin again,acoustic
3,kinagrannis,crazy rich asians (original motion picture sou...,can't help falling in love,acoustic
4,chordoverstreet,hold on,hold on,acoustic


In [12]:

# Create a new 'tags' column
df['tags'] = df['artists'] + " " + df['album_name'] + " " + df['track_genre']

In [13]:
music = df[['track_name', 'tags']]

In [14]:
music.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113079 entries, 0 to 113999
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   track_name  113079 non-null  object
 1   tags        113079 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


In [15]:
music.head()

,track_name,tags
0,comedy,genhoshino comedy acoustic
1,ghost - acoustic,benwoodward ghost (acoustic) acoustic
2,to begin again,ingridmichaelson;zayn to begin again acoustic
3,can't help falling in love,kinagrannis crazy rich asians (original motion...
4,hold on,chordoverstreet hold on acoustic


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
# Vectorize the 'tags' column
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(music['tags']).toarray()
print(vectors.shape)

(113079, 5000)


In [18]:

from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(vectors)

NearestNeighbors(algorithm='brute', metric='cosine')

In [19]:
def recommendation(track_name):
    if track_name not in df['track_name'].values:
        print("Track not found.")
        return
    track_index = df[df['track_name'] == track_name].index[0]
    distances, indices = nn.kneighbors([vectors[track_index]], n_neighbors=6)
    for i in indices[0][1:]:
        print(df.iloc[i]['track_name'])


In [37]:
recommendation('milne hai mujhse aayi')

is qadar
paniyon sa
yeh sham mastani
genda phool
venmathiye
